In [ ]:
# |default_exp mongo

# DB variables

In [ ]:
# |export

# to fix the forward reference issue by from_oid, get_entries in BaseEntity
from __future__ import annotations

from dotenv import load_dotenv, find_dotenv
import os
from typing import Tuple, Optional


def load_env() -> None:
    """Load environment variables from a .env file.

    This function searches for a .env file in the project directory and loads its
    variables into the environment.
    """
    _ = load_dotenv(find_dotenv())


def get_DB_VARS() -> Tuple[Optional[str], Optional[str], Optional[str]]:
    """Retrieve the database configuration variables from the environment.

    This function loads the environment variables and retrieves the following
    database configuration variables:
        - DB_HOST: The hostname for the database.
        - DB_NAME: The name of the database.
        - DB_LOGS: A flag indicating if logging is enabled.

    Returns:
        Tuple[Optional[str], Optional[str], Optional[str]]:
            A tuple containing (DB_HOST, DB_NAME, DB_LOGS).
    """
    load_env()
    DB_HOST: Optional[str] = os.getenv("DB_HOST")
    DB_NAME: Optional[str] = os.getenv("DB_NAME")
    DB_LOGS: Optional[str] = os.getenv("DB_LOGS")
    return DB_HOST, DB_NAME, DB_LOGS

# get collection

In [ ]:
# |export

import pymongo
from pymongo.collection import Collection


def get_collection(
    target_db: str = "localhost",
    client_name: str = "masque_et_la_plume",
    collection_name: str = "episodes",
) -> Collection:
    """Retrieve a MongoDB collection.

    This function connects to a MongoDB database using the provided database host,
    client name (database name), and collection name, and returns the collection object.

    Args:
        target_db (str): The database host address (e.g., "localhost" or "nas923").
        client_name (str): The name of the MongoDB client/database.
        collection_name (str): The name of the collection to retrieve.

    Returns:
        Collection: The MongoDB collection object.
    """
    client = pymongo.MongoClient(f"mongodb://{target_db}:27017/")
    db = client[client_name]
    collection = db[collection_name]
    return collection

# logs

In [ ]:
# |export

from datetime import datetime
import pymongo


def mongolog(operation: str, entite: str, desc: str) -> None:
    """Enregistre une opération de log dans la collection 'logs' si la configuration autorise les logs.

    Args:
        operation (str): L'opération effectuée (par exemple, "insert", "update", "delete").
        entite (str): Le nom de l'entité concernée.
        desc (str): Une description détaillée de l'opération.
    """
    DB_HOST, DB_NAME, DB_LOGS = get_DB_VARS()
    if DB_LOGS in ["true", "True"]:
        coll_logs = get_collection(DB_HOST, DB_NAME, "logs")
        coll_logs.insert_one(
            {
                "operation": operation,
                "entite": entite,
                "desc": desc,
                "date": datetime.now(),
            }
        )


def print_logs(n: int = 10) -> None:
    """Affiche les n derniers logs de la collection 'logs', triés par date décroissante.

    Args:
        n (int, optional): Le nombre maximum de logs à afficher. Par défaut à 10.
    """
    DB_HOST, DB_NAME, DB_LOGS = get_DB_VARS()
    coll_logs = get_collection(DB_HOST, DB_NAME, "logs")
    for i, log in enumerate(coll_logs.find().sort("date", pymongo.DESCENDING)):
        if i == n:
            break
        print(log)

In [ ]:
print_logs(5)

{'_id': ObjectId('679a68382b212aee21955fc2'), 'operation': 'force_update', 'entite': 'episodes', 'desc': '16 nov. 1961 00:00 - Le Masque et les Goncourt : "L\'Etat sauvage" de Georges Conchon, 1964 -> 18 nov. 1964 00:00', 'date': datetime.datetime(2025, 1, 29, 18, 41, 12, 512000)}
{'_id': ObjectId('679a68382b212aee21955fbf'), 'operation': 'force_update', 'entite': 'episodes', 'desc': '11 sept. 2016 00:00 - Le Masque et les Goncourt : "Chanson douce" de Leïla Slimani, 2016 -> 12 sept. 2016 00:00', 'date': datetime.datetime(2025, 1, 29, 18, 41, 12, 503000)}
{'_id': ObjectId('679a68382b212aee21955fbc'), 'operation': 'force_update', 'entite': 'episodes', 'desc': '02 juil. 2017 00:00 - Le Masque et les Goncourt : "L\'Ordre du jour" d\'Eric Vuillard, 2017 -> 03 juil. 2017 00:00', 'date': datetime.datetime(2025, 1, 29, 18, 41, 12, 491000)}
{'_id': ObjectId('679a68382b212aee21955fb9'), 'operation': 'force_update', 'entite': 'episodes', 'desc': '25 nov. 2018 00:00 - Le Masque et les Goncourt : 

# Base entity

c'est une classe de base issue de la refactorisation de Auteur et Editeur proposée par github copilot


> Étapes de Raisonnement pour la Factorisation
> Analyser les Classes Existantes :
> 
> Les classes Auteur et Editeur ont des méthodes similaires pour vérifier l'existence, insérer, mettre à jour et supprimer des documents dans la base de données MongoDB.
> Les méthodes exists, keep, remove, et get_oid sont communes aux deux classes.
> Identifier les Comportements Communs :
> 
> Les comportements communs peuvent être regroupés dans une classe de base.
> Les classes spécifiques (Auteur et Editeur) peuvent hériter de cette classe de base et ajouter des comportements spécifiques si nécessaire.
> Créer une Classe de Base :
> 
> Créer une classe de base BaseEntity qui contient les méthodes communes.
> La classe de base gérera les interactions avec la base de données.
> Hériter de la Classe de Base :
> 
> Les classes Auteur et Editeur hériteront de BaseEntity.
> Elles définiront uniquement les comportements spécifiques à chaque entité.



In [ ]:
# |export

from bson import ObjectId
from typing import List, Optional, Type, TypeVar

T = TypeVar("T", bound="BaseEntity")


class BaseEntity:
    def __init__(self, nom: str, collection_name: str) -> None:
        """Initializes a new BaseEntity instance.

        Args:
            nom (str): The name of the entity.
            collection_name (str): The name of the collection.
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        self.collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=collection_name
        )
        self.nom = nom

    def exists(self) -> bool:
        """Checks if the entity exists in the database.

        Returns:
            bool: True if the entity exists, False otherwise.
        """
        return self.collection.find_one({"nom": self.nom}) is not None

    def keep(self) -> None:
        """Inserts or updates the entity in the database.

        If the entity does not exist, it is inserted; otherwise, it is updated.
        """
        if not self.exists():
            mongolog("insert", self.collection.name, self.nom)
            self.collection.insert_one({"nom": self.nom})
        else:
            mongolog("update", self.collection.name, self.nom)

    def remove(self) -> None:
        """Removes the entity from the database."""
        self.collection.delete_one({"nom": self.nom})
        mongolog("delete", self.collection.name, self.nom)

    def get_oid(self) -> Optional[ObjectId]:
        """Retrieves the ObjectId of the entity from the database.

        Returns:
            Optional[ObjectId]: The ObjectId of the entity if found, otherwise None.
        """
        document = self.collection.find_one({"nom": self.nom})
        if document:
            return document["_id"]
        else:
            return None

    @classmethod
    def from_oid(cls: Type[T], oid: ObjectId) -> T:
        """Creates an instance of the derived class from a MongoDB ObjectId.

        Args:
            oid (ObjectId): The MongoDB ObjectId.

        Returns:
            T: An instance of the derived class.
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=cls.collection
        )
        document = collection.find_one({"_id": oid})
        inst = cls(nom=document.get("nom"))
        return inst

    @classmethod
    def get_entries(cls: Type[T], request: str = "") -> List[T]:
        """Retrieves a list of entries matching the query.

        Args:
            request (str, optional): A substring of the name to filter results
                (case-insensitive). Defaults to an empty string.

        Returns:
            List[T]: A list of instances of the derived class.
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=cls.collection
        )
        query = {
            "nom": {
                "$regex": request,
                "$options": "i",
            }
        }
        result = collection.find(query)
        list_baseentity = [cls.from_oid(entry.get("_id")) for entry in result]
        return list_baseentity

    def __repr__(self) -> str:
        """Official string representation of the entity.

        Returns:
            str: The name of the entity.
        """
        return self.nom

    def __str__(self) -> str:
        """Informal string representation of the entity.

        Returns:
            str: The name of the entity.
        """
        return self.nom

# Editeur

In [ ]:
# |export
class Editeur(BaseEntity):
    """
    Represents a publisher stored in the 'editeurs' MongoDB collection.

    Attributes:
        collection (str): The name of the MongoDB collection.
    """

    collection: str = "editeurs"

    def __init__(self, nom: str) -> None:
        """
        Initialize an Editeur instance.

        Args:
            nom (str): The name of the publisher.
        """
        super().__init__(nom, self.collection)

In [ ]:
Editeur.get_entries()

[]

In [ ]:
editeur1 = Editeur("Gallimard")
print(f"Est-ce que editeur1 existe ? {editeur1.exists()}")

editeur1.keep()
print(f"et maintenant, st-ce que editeur1 existe ? {editeur1.exists()}")
print(f"et voici l'id de editeur1 : {editeur1.get_oid()}")

editeur1.remove()
print(f"après nettoyage, est-ce que editeur1 existe ? {editeur1.exists()}")
print(f"et son oid : {editeur1.get_oid()}")

Est-ce que editeur1 existe ? False
et maintenant, st-ce que editeur1 existe ? True
et voici l'id de editeur1 : 67a521e460b4498ed3f3262d
après nettoyage, est-ce que editeur1 existe ? False
et son oid : None


In [ ]:
print_logs(5)

{'_id': ObjectId('67a521e460b4498ed3f3262f'), 'operation': 'delete', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 2, 6, 21, 56, 4, 808000)}
{'_id': ObjectId('67a521e460b4498ed3f3262c'), 'operation': 'insert', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 2, 6, 21, 56, 4, 799000)}
{'_id': ObjectId('67a521ad60b4498ed3f32624'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 2, 6, 21, 55, 9, 78000)}
{'_id': ObjectId('67a521ad60b4498ed3f32621'), 'operation': 'insert', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 2, 6, 21, 55, 9, 64000)}
{'_id': ObjectId('679a68382b212aee21955fc2'), 'operation': 'force_update', 'entite': 'episodes', 'desc': '16 nov. 1961 00:00 - Le Masque et les Goncourt : "L\'Etat sauvage" de Georges Conchon, 1964 -> 18 nov. 1964 00:00', 'date': datetime.datetime(2025, 1, 29, 18, 41, 12, 512000)}


# Critique

In [ ]:
# |export


class Critique(BaseEntity):
    """
    Class representing a review (Critique) entity stored in the 'critiques' MongoDB collection.

    Attributes:
        collection (str): MongoDB collection name used to store critiques.
    """

    collection: str = "critiques"

    def __init__(self, nom: str) -> None:
        """
        Initializes a Critique instance.

        Args:
            nom (str): Name of the critique.
        """
        super().__init__(nom, self.collection)

In [ ]:
Critique.get_entries()

[]

In [ ]:
critique1 = Critique("Patricia Martin")
print(f"Est-ce que critique1 existe ? {critique1.exists()}")

critique1.keep()
print(f"et maintenant, st-ce que critique1 existe ? {critique1.exists()}")
print(f"et voici l'id de critique1 : {critique1.get_oid()}")

critique1.remove()
print(f"après nettoyage, est-ce que critique1 existe ? {critique1.exists()}")
print(f"et son oid : {critique1.get_oid()}")

Est-ce que critique1 existe ? False
et maintenant, st-ce que critique1 existe ? True
et voici l'id de critique1 : 67a521ec60b4498ed3f32635
après nettoyage, est-ce que critique1 existe ? False
et son oid : None


In [ ]:
print_logs(5)

{'_id': ObjectId('67a521ec60b4498ed3f32637'), 'operation': 'delete', 'entite': 'critiques', 'desc': 'Patricia Martin', 'date': datetime.datetime(2025, 2, 6, 21, 56, 12, 93000)}
{'_id': ObjectId('67a521ec60b4498ed3f32634'), 'operation': 'insert', 'entite': 'critiques', 'desc': 'Patricia Martin', 'date': datetime.datetime(2025, 2, 6, 21, 56, 12, 83000)}
{'_id': ObjectId('67a521e460b4498ed3f3262f'), 'operation': 'delete', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 2, 6, 21, 56, 4, 808000)}
{'_id': ObjectId('67a521e460b4498ed3f3262c'), 'operation': 'insert', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 2, 6, 21, 56, 4, 799000)}
{'_id': ObjectId('67a521ad60b4498ed3f32624'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 2, 6, 21, 55, 9, 78000)}


# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("py mongo helper.ipynb", ".")